In [10]:
class RestorableState:
    def __init__(self, context, state, cm_raises_exception):
        self.state = state
        self.restore_state = None
        self.context = context
        self.cm_raises_exception = cm_raises_exception
        
    def __enter__(self):
        if self.cm_raises_exception:
            ex, *args = self.cm_raises_exception
            raise ex(*args)
        return self.context

    def __exit__(self, *args):
        self.state = self.restore_state
        return False

#     def __call__(self, *args):
#         self.restore_state = self.state
#         self.state = args
#         return self #.color

    def __repr__(self):
        return f"{self.__class__.__name__}: {self.state}"

    
def normalize_color(args):
    if args is None:
        return None
    
    if len(args) == 1:
        c = args[0]
        return (c, c, c, 1)
    elif len(args) == 2:
        return (args[0], args[1], 1, 1)
    elif len(args) == 3:
        return (*args, 1)
    else:
        return args

    
# class ColorState(RestorableState):
#     def __init__(self, context, color, cm_raises_exception=None):
#         RestorableState.__init__(self, context, color, cm_raises_exception=cm_raises_exception)
        
#     def __call__(self, *args):
#         color = normalize_color(args)
#         return super().__call__(color)


class ColorState(RestorableState):
    def __init__(self, context, color, cm_raises_exception=None):
        RestorableState.__init__(self, context, color, cm_raises_exception=cm_raises_exception)
        
#     def __call__(self, *args):
#         color = normalize_color(args)
#         return super().__call__(*color)
    

class Grob:
    pass

class Color(Grob):
    def __init__(self, context, *args):
        self._state = ColorState(context, normalize_color(args))
        
    def __enter__(self):
        return Color()

IDENTITY_MATRIX = [
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
]
    
class Transform(RestorableState):
    def __init__(self, context, matrix=None):
        if matrix is None:
            matrix = IDENTITY_MATRIX
        RestorableState.__init__(self, context, matrix, cm_raises_exception=False)
        
    def skew(self, x=0, y=0):
        if (x, y) == (0, 0):
            return self
        return self
    
    def rotate(self, degrees):
        return self
    
    def push(self, t):
        self.__call__(t)
        return self.__enter__()
    
    def pop(self):
        return self.__exit__()

class Context:
    def __init__(self):
        self._fill = Color(self, (1, 1, 1, 1))
        self._stroke = Color(self, (0, 0, 0, 1))
        self._transform = Transform(self)
        
    def fill(self, *args):
        return Color(self, *args)
        
#     def move(self, x, y):
#         self.transform.move_to(x, y)
    
#     def push(self):
#         return self.transform.push()
    
#     def pop(self):
#         return self.transform.pop()
    
#     def skew(self, *args):
#         return self.transform.skew(*args)
        
        
ctx = Context()
#ctx.fill()

# print(dir(ctx))
# with ctx.fill(0):
#     pass
#print(dir(ctx.fill()))

ctx.fill(.1, .1, .1, 1)

print(ctx.fill)
with ctx.fill(1, 0, 0, 1), ctx.stroke(0):
    print(ctx.fill, ctx.stroke)
    with ctx.skew(45):
        print(ctx.transform)
    
print(ctx.fill)

ctx.fill()

<bound method Context.fill of <__main__.Context object at 0x7faf80552280>>


AttributeError: __exit__